In [43]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from time import sleep
import pandas as pd
from collections import defaultdict 

import csv
from tqdm import tqdm
from pathlib import Path

In [44]:
path = Path.cwd()
file = pd.read_excel(path/'keyword.xlsx', engine = 'openpyxl')
category_list = file.values.tolist()

In [45]:
column = ['등록번호','참조번호','품명(사업명)','수요기관','사전규격공개일시','업체등록의견수']
df = pd.DataFrame(columns=column)
try:
    driver = webdriver.Chrome('./chromedriver')
    driver.implicitly_wait(2)

    for query in category_list:

        
        
        driver.get('https://www.g2b.go.kr:8081/ep/preparation/prestd/preStdSrch.do')
        task_dict = {'용역': 'taskClCds5'}
        for task in task_dict.values():
            checkbox = driver.find_element(By.ID, task)
            checkbox.click()
        # ID값이 prodNm인 태그 가져오기
        prodNm = driver.find_element(By.ID, 'prodNm')
        # 내용을 삭제
        prodNm.clear()
        # 검색어에 문자열 전달
        prodNm.send_keys(query)
        prodNm.send_keys(Keys.RETURN)
        
        # 목록수 100건 선택 (드롭다운)
        print(1)
        recordcountperpage = driver.find_element(By.NAME,'recordCountPerPage')
        selector = Select(recordcountperpage)
        selector.select_by_value('100')

        search_button = driver.find_element(By.CLASS_NAME, 'btn_mdl')
        search_button.click()
        print(2)
        # 검색 결과 확인
        elem = driver.find_element(By.CLASS_NAME,'results')
        div_list = elem.find_elements(By.TAG_NAME, 'tr')

        # 리스트 형태로 검색결과 저장
        
        print(3)
        results = []
        links = []
        for div in tqdm(div_list):
            results.append(div.text)
            row_data = [div.text.split('\n')]
            a_tags = div.find_elements(By.TAG_NAME,'a')
            if a_tags:
                a_tag = a_tags[0]
                link = a_tag.get_attribute('href')
                row_data.append([link])
            # create a new DataFrame with the data from this row
            
            df_row = pd.DataFrame(row_data)
            # append it to the overall df using pd.concat()
            df = pd.concat([df, df_row], ignore_index=True)
                #df = df.assign(링크 =links)

except Exception as e:
    print(e)
finally:  
    driver.quit()

C:\Users\JOONHY~1\AppData\Local\Temp/ipykernel_26616/1267538500.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


1
2
3


100%|██████████| 101/101 [00:09<00:00, 10.64it/s]


1
2
3


100%|██████████| 35/35 [00:04<00:00,  7.01it/s]


In [46]:
df

,등록번호,참조번호,품명(사업명),수요기관,사전규격공개일시,업체등록의견수,0,1,2,3,4,5,6
0,NaN,NaN,NaN,NaN,NaN,NaN,No. 등록번호 참조번호 품명(사업명) 수요기관 사전규격공개일시 업체등록의견수,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,1,1271656,미래스마트건설연구본부-2989,이동 플랫폼 영상 데이터 위치정보 생산을 위한 프로토타입 모듈 개발,한국건설기술연구원,2023/03/24 17:33,0
2,NaN,NaN,NaN,NaN,NaN,NaN,javascript:toDtl('1271656');,None,None,None,None,None,None
3,NaN,NaN,NaN,NaN,NaN,NaN,2,1271634,미래스마트건설연구본부-2941,교량 손상 객체 자동식별 소프트웨어 확장 및 이미지 메타 데이터를 활용한 정량화 모...,한국건설기술연구원,2023/03/24 17:23,0
4,NaN,NaN,NaN,NaN,NaN,NaN,javascript:toDtl('1271634');,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,NaN,NaN,NaN,NaN,NaN,NaN,javascript:toDtl('1256763');,None,None,None,None,None,None
266,NaN,NaN,NaN,NaN,NaN,NaN,33,1253293,도시재생과-2464,남양주시 도시재생 전략계획(변경) 및 진접읍 활성회계획 수립 용역[협상에 의한 계약],경기도 남양주시,2023/02/23 16:08,0
267,NaN,NaN,NaN,NaN,NaN,NaN,javascript:toDtl('1253293');,None,None,None,None,None,None
268,NaN,NaN,NaN,NaN,NaN,NaN,34,1254926,교통과-11605,2022년도 표준운송원가 산정 및 회계감사 용역,세종특별자치시,2023/02/22 18:30,0


In [47]:
column = ['등록번호','참조번호','품명(사업명)','수요기관','사전규격공개일시','업체등록의견수']
new_df = df.iloc[:,7:14]
new_df.columns = column

In [48]:
new_df = new_df.dropna(subset=['등록번호'])
new_df = new_df.reset_index(drop=True)
new_df['링크'] = 0
new_df = new_df.drop_duplicates(['등록번호'])
for i in range(len(new_df)):
    new_df['링크'][i] = 'https://www.g2b.go.kr:8082/ep/preparation/prestd/preStdDtl.do?preStdRegNo=' + new_df['등록번호'][i]


C:\Users\JOONHY~1\AppData\Local\Temp/ipykernel_26616/2726599511.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['링크'][i] = 'https://www.g2b.go.kr:8082/ep/preparation/prestd/preStdDtl.do?preStdRegNo=' + new_df['등록번호'][i]


In [49]:
from datetime import datetime 
today = datetime.today().strftime('%Y-%m-%d')

In [50]:
new_df.to_excel(f'RA_Operational_공고_{today}.xlsx')

In [51]:
new_df

,등록번호,참조번호,품명(사업명),수요기관,사전규격공개일시,업체등록의견수,링크
0,1271656,미래스마트건설연구본부-2989,이동 플랫폼 영상 데이터 위치정보 생산을 위한 프로토타입 모듈 개발,한국건설기술연구원,2023/03/24 17:33,0,https://www.g2b.go.kr:8082/ep/preparation/pres...
1,1271634,미래스마트건설연구본부-2941,교량 손상 객체 자동식별 소프트웨어 확장 및 이미지 메타 데이터를 활용한 정량화 모...,한국건설기술연구원,2023/03/24 17:23,0,https://www.g2b.go.kr:8082/ep/preparation/pres...
2,1271610,구조연구본부-2569,해상특수교량 주탑부 드론 촬영 및 영상데이터 분석 용역,한국건설기술연구원,2023/03/24 17:05,0,https://www.g2b.go.kr:8082/ep/preparation/pres...
3,1271606,미래스마트건설연구본부-2814,건축물 안전정보 통합 활용을 위한 sBIM 데이터와 실내공간정보 표준 데이터 융합 ...,한국건설기술연구원,2023/03/24 17:01,0,https://www.g2b.go.kr:8082/ep/preparation/pres...
4,1271367,정보기획팀-10373,공공데이터 평가 및 데이터기반행정 실태점검 역량강화 컨설팅,한국연구재단,2023/03/24 14:00,0,https://www.g2b.go.kr:8082/ep/preparation/pres...
...,...,...,...,...,...,...,...
129,1258187,BF2023018,'23~'25년 회계연도 외부회계감사용역,주식회사 에스알,2023/03/02 08:19,0,https://www.g2b.go.kr:8082/ep/preparation/pres...
130,1257985,공고 경영지원실-2023-제52호,2023~2025 회계연도 회계감사 및 세무조정,국민건강보험공단,2023/02/28 17:53,0,https://www.g2b.go.kr:8082/ep/preparation/pres...
131,1256763,한국언론진흥재단 제2023-66호,2022회계연도 한국언론진흥재단 법인회계 예산결산 검증 용역,한국언론진흥재단,2023/02/27 10:36,0,https://www.g2b.go.kr:8082/ep/preparation/pres...
132,1253293,도시재생과-2464,남양주시 도시재생 전략계획(변경) 및 진접읍 활성회계획 수립 용역[협상에 의한 계약],경기도 남양주시,2023/02/23 16:08,0,https://www.g2b.go.kr:8082/ep/preparation/pres...
